# MOABB Classifier comparison

This notebook takes multiple files and does the following pipeline:
1) Import data and pre-process
2) For each stimulus type 
    1) Run classifiers (i.e., CCA, MEC, MSI, and RG)
    2) Store accuracy values

A final plot for each stimulus type is plotted 

In [1]:
# Default libraries
import re
import mne
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline

from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace

from moabb.datasets import Wang2016, SSVEPExo
from moabb.paradigms import FilterBankSSVEP, SSVEP
from moabb.pipelines import ExtendedSSVEPSignal
from sklearn.linear_model import LogisticRegression
from moabb.evaluations import CrossSubjectEvaluation

In [2]:
# Import custom libraries
from ncantest import data_tools
from ncantest import processing
from ncantest import classification

In [3]:
from FeatureExtractorSSVEP import FeatureExtractorCCA as CCA
from FeatureExtractorSSVEP import FeatureExtractorMSI as MSI
from FeatureExtractorSSVEP import FeatureExtractorMEC as MEC

# Magic command to reload libraries
%reload_ext autoreload

## Settings

Note that if the `Wang2016` is not already downloaded, it'll be downloaded the first time you run this section of the code.

In [4]:
mne.set_config("MNE_DATA", "~/.mne")

In [5]:
mne.get_config("MNE_DATA")

'~/.mne'

In [6]:
# Import and epoch data
dataset = Wang2016()

In [7]:
# Limit to 2 subjects to speed up during testing
dataset.subject_list = dataset.subject_list[:2]

In [8]:
data = dataset.get_data()

Trial data de-meaned and concatenated with a buffer to create continuous data
Trial data de-meaned and concatenated with a buffer to create continuous data


In [9]:
eeg_channels = ["O1","Oz","O2"]

# Information from dataset description
nsubjects = len(data)
# labels_dict = {"13":0, "17":2, "21":1}  # The order is changed because the labels in the dataset are incorrect
# stimulus_freqs = [int(f) for f in labels_dict.keys()]

stimulus_freqs = [float(freq) for freq in dataset.event_id.keys()]  # Stimulus frequencies [Hz]
srate = data[1]["0"]["0"].info['sfreq'] # Sampling rate [Hz]
tmin = 0.5  # Time of start of SSVEP stimulus [sec]
tmax = 5.5  # Time of end of SSVEP stimulus [sec]

# Classifier settings
# - Create CCA subbands like in Chen et al. (2015) paper
first_column = np.arange(1, 11) * 8
second_column = np.full(10, 88)
cca_subbands = np.column_stack((first_column, second_column))
harmonic_count = 2
classifiers = ["fbCCA", "MSI", "MEC", "RG_logreg"]

# Create an empty dataframe to store the accuracies
accuracy_df = pd.DataFrame(
    index = np.arange(0, nsubjects),
    columns = classifiers
    )
accuracy_df.index.name = "Subject"

## Separate epochs

From the raw recording datasets, create the EEG epochs using just the periods where the SSVEP stimulus was active.

In [10]:
# Preallocate data
epochs_list = [None] * len(data)
events_list = [None] * len(data)

# Obtain epochs and events
for s, subject in data.items():
    [events_list[s-1], epochs_list[s-1]] = data_tools.moabb_events_to_np(
        mne_raw = subject["0"]["0"],
        tmin = tmin,
        tmax = tmax,
        events_dict = dataset.event_id,
        chans = eeg_channels
        )
    
# Convert lists to np.ndarrays
# eeg_channels = data[1]["0"]["0"].ch_names
epochs_np = np.float32(np.array(epochs_list))
events_np = np.array(events_list[0][:,2]) - 1   # The `-1` is to make the labels start at 0

## Classifiers

### fbCCA

In [11]:
# Prototype and preallocate data
cca = CCA()
cca_accuracies = np.zeros(nsubjects)

In [12]:
cca.setup_feature_extractor(
    harmonics_count = harmonic_count,
    targets_frequencies = stimulus_freqs,
    sampling_frequency = srate,
    samples_count = epochs_np.shape[-1],
    filter_order = 12,
    subbands = cca_subbands
    )
    

Platform 'METAL' is experimental and not all JAX functionality may be correctly supported!


Metal device set to: Apple M2 Max


W0000 00:00:1732577777.466404 2768448 mps_client.cc:510] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!
I0000 00:00:1732577777.476803 2768448 service.cc:145] XLA service 0x600003d9d000 initialized for platform METAL (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732577777.476821 2768448 service.cc:153]   StreamExecutor device (0): Metal, <undefined>
I0000 00:00:1732577777.478473 2768448 mps_client.cc:406] Using Simple allocator.
I0000 00:00:1732577777.478490 2768448 mps_client.cc:384] XLA backend will use up to 77309018112 bytes on device 0 for SimpleAllocator.


In [13]:
for s, subject in enumerate(epochs_np):
    print(s)
    # print(subject)

0
1


In [15]:
cca_features = cca.extract_features(subject)

In [17]:
cca_features.shape

(240, 1, 1, 40, 1)

In [18]:
cca_features[1].shape

(1, 1, 40, 1)

In [19]:
a=np.squeeze(cca_features)
a.shape

(240, 40)

In [20]:
b=np.max(a, axis=1)
b.shape

(240,)

In [21]:
c=np.argmax(b, axis=1)
c.shape

IndexError: tuple index out of range

In [22]:
c

NameError: name 'c' is not defined

In [ ]:
type(cca_features)

In [ ]:
cca_features.shape

In [ ]:
cca_features.size

In [ ]:
numpy_array = np.array(cca_features)

In [ ]:
numpy_array.shape

In [ ]:
np.argmax(np.max(np.squeeze(numpy_array), axis=1), axis=1)

In [ ]:
a=np.squeeze(numpy_array)

In [ ]:
a.shape

In [ ]:
np.max(a)

In [ ]:
b=np.max(a, axis=1)

In [ ]:
b.shape

In [ ]:
b

In [ ]:
epochs_list[0].shape

In [ ]:
b.shape

In [ ]:
len(epochs_list)

In [ ]:
subject.shape

In [ ]:
cca_features.shape

In [ ]:
events_np.shape

In [ ]:
np.argmax

In [ ]:
cca_predictions = np.max(np.squeeze(cca_features), axis=1)

In [ ]:
cca_predictions.shape

In [ ]:
cca_accuracies[s] = accuracy_score(events_np, cca_predictions)

In [ ]:
events_np

In [ ]:
print(np.argmax(cca_predictions))

In [ ]:
# Classify all epochs per subject
for s, subject in enumerate(epochs_np):
    cca_features = cca.extract_features(subject)
    cca_predictions = np.argmax(np.max(np.squeeze(cca_features), axis=1), axis=1)
    cca_accuracies[s] = accuracy_score(events_np, cca_predictions)

In [ ]:

accuracy_df["fbCCA"] = cca_accuracies
    

### MEC

### MSI

### Riemmanian geometry + logistic regression
**IGNORE – FROM MOABB**

## Visualization

### Boxplot

## Save results
